In [36]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import time
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

In [37]:
footballer_df = pd.read_csv('footballer_stats_from_R.csv', low_memory = False)

In [38]:
price_value = pd.read_csv("transfermarkt_price.csv")

In [39]:
footballer_df.rename(columns={'Season_End_Year_x': 'Season'}, inplace=True)
footballer_df.rename(columns={'Player_x': 'Name'}, inplace=True)

In [40]:
footballer_df['Season'] = footballer_df['Season'] -1 

In [41]:
price_value.rename(columns={'Year': 'Season'}, inplace=True)

In [42]:
conditions = ['Season','Name']

# Fuzzy Technique dan pake how = ' outer'

In [43]:
test_df= pd.merge(footballer_df, price_value , on=conditions, how = 'outer')
test_df

,Season,Squad_x,Comp_x,Name,Nation_x,Pos_x,Age_x,Born_x,MP_Playing,Starts_Playing,...,CPA_Carries,Mis_Carries,Dis_Carries,Rec_Receiving,PrgR_Receiving,Url,Squad_Year,Rk,Value,Position
0,2017,Alavés,La Liga,Martin Agirregabiria,ESP,DF,21.0,1996.0,21.0,20.0,...,6.0,20.0,10.0,483.0,54.0,https://fbref.com/en/players/355c883a/Martin-A...,2018Alavés,14.0,NaN,NaN
1,2017,Alavés,La Liga,Alexis,ESP,DF,31.0,1985.0,28.0,25.0,...,2.0,8.0,3.0,389.0,11.0,https://fbref.com/en/players/db0cd1ae/Alexis,2018Alavés,14.0,NaN,NaN
2,2017,Alavés,La Liga,Burgui,ESP,"MF,FW",23.0,1993.0,23.0,12.0,...,14.0,23.0,17.0,348.0,75.0,https://fbref.com/en/players/45e304c8/Burgui,2018Alavés,14.0,€1.80m,Left Winger
3,2017,Alavés,La Liga,Ermedin Demirović,BIH,FW,19.0,1998.0,3.0,2.0,...,2.0,4.0,4.0,42.0,11.0,https://fbref.com/en/players/ed79b7d3/Ermedin-...,2018Alavés,14.0,NaN,NaN
4,2017,Alavés,La Liga,Adrián Diéguez,ESP,DF,21.0,1996.0,6.0,6.0,...,1.0,3.0,3.0,103.0,3.0,https://fbref.com/en/players/70438d35/Adrian-D...,2018Alavés,14.0,€400k,Centre-Back
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34202,2017,NaN,NaN,Roy Tol,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,€50k,Right Winger
34203,2017,NaN,NaN,Enzo Stroo,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,€250k,Centre-Forward
34204,2017,NaN,NaN,Teije ten Den,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,€150k,Centre-Forward
34205,2017,NaN,NaN,Jordy Hilterman,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,€50k,Centre-Forward


In [44]:
test_df_no_price = test_df.dropna(subset=['Value'])
test_df_no_price

,Season,Squad_x,Comp_x,Name,Nation_x,Pos_x,Age_x,Born_x,MP_Playing,Starts_Playing,...,CPA_Carries,Mis_Carries,Dis_Carries,Rec_Receiving,PrgR_Receiving,Url,Squad_Year,Rk,Value,Position
2,2017,Alavés,La Liga,Burgui,ESP,"MF,FW",23.0,1993.0,23.0,12.0,...,14.0,23.0,17.0,348.0,75.0,https://fbref.com/en/players/45e304c8/Burgui,2018Alavés,14.0,€1.80m,Left Winger
4,2017,Alavés,La Liga,Adrián Diéguez,ESP,DF,21.0,1996.0,6.0,6.0,...,1.0,3.0,3.0,103.0,3.0,https://fbref.com/en/players/70438d35/Adrian-D...,2018Alavés,14.0,€400k,Centre-Back
5,2017,Alavés,La Liga,Rubén Duarte,ESP,DF,21.0,1995.0,24.0,24.0,...,2.0,12.0,13.0,466.0,48.0,https://fbref.com/en/players/1fe0492d/Ruben-Du...,2018Alavés,14.0,€2.00m,Left-Back
6,2017,Alavés,La Liga,Rodrigo Ely,BRA,DF,23.0,1993.0,31.0,30.0,...,0.0,9.0,3.0,635.0,7.0,https://fbref.com/en/players/fd47b4f2/Rodrigo-Ely,2018Alavés,14.0,€6.00m,Centre-Back
7,2017,Alavés,La Liga,Manu García,ESP,MF,31.0,1986.0,30.0,27.0,...,1.0,28.0,28.0,761.0,48.0,https://fbref.com/en/players/05a4465d/Manu-Garcia,2018Alavés,14.0,€1.80m,Central Midfield
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34202,2017,NaN,NaN,Roy Tol,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,€50k,Right Winger
34203,2017,NaN,NaN,Enzo Stroo,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,€250k,Centre-Forward
34204,2017,NaN,NaN,Teije ten Den,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,€150k,Centre-Forward
34205,2017,NaN,NaN,Jordy Hilterman,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,€50k,Centre-Forward


In [45]:
test_df_no_price_name = test_df_no_price.dropna(subset=['Squad_x'])
test_df_no_price_name

,Season,Squad_x,Comp_x,Name,Nation_x,Pos_x,Age_x,Born_x,MP_Playing,Starts_Playing,...,CPA_Carries,Mis_Carries,Dis_Carries,Rec_Receiving,PrgR_Receiving,Url,Squad_Year,Rk,Value,Position
2,2017,Alavés,La Liga,Burgui,ESP,"MF,FW",23.0,1993.0,23.0,12.0,...,14.0,23.0,17.0,348.0,75.0,https://fbref.com/en/players/45e304c8/Burgui,2018Alavés,14.0,€1.80m,Left Winger
4,2017,Alavés,La Liga,Adrián Diéguez,ESP,DF,21.0,1996.0,6.0,6.0,...,1.0,3.0,3.0,103.0,3.0,https://fbref.com/en/players/70438d35/Adrian-D...,2018Alavés,14.0,€400k,Centre-Back
5,2017,Alavés,La Liga,Rubén Duarte,ESP,DF,21.0,1995.0,24.0,24.0,...,2.0,12.0,13.0,466.0,48.0,https://fbref.com/en/players/1fe0492d/Ruben-Du...,2018Alavés,14.0,€2.00m,Left-Back
6,2017,Alavés,La Liga,Rodrigo Ely,BRA,DF,23.0,1993.0,31.0,30.0,...,0.0,9.0,3.0,635.0,7.0,https://fbref.com/en/players/fd47b4f2/Rodrigo-Ely,2018Alavés,14.0,€6.00m,Centre-Back
7,2017,Alavés,La Liga,Manu García,ESP,MF,31.0,1986.0,30.0,27.0,...,1.0,28.0,28.0,761.0,48.0,https://fbref.com/en/players/05a4465d/Manu-Garcia,2018Alavés,14.0,€1.80m,Central Midfield
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17884,2023,Wolves,Premier League,Pedro Neto,POR,"FW,MF",24-004,2000.0,19.0,18.0,...,42.0,44.0,20.0,673.0,148.0,https://fbref.com/en/players/7ba2eaa9/Pedro-Neto,2024Wolves,9.0,€55.00m,Right Winger
17885,2023,Wolves,Premier League,José Sá,POR,GK,31-056,1993.0,26.0,26.0,...,1.0,0.0,1.0,328.0,0.0,https://fbref.com/en/players/903b6e8b/Jose-Sa,2024Wolves,9.0,€16.00m,Goalkeeper
17886,2023,Wolves,Premier League,Pablo Sarabia,ESP,"MF,FW",31-307,1992.0,21.0,15.0,...,8.0,28.0,16.0,605.0,105.0,https://fbref.com/en/players/9744ff80/Pablo-Sa...,2024Wolves,9.0,€12.00m,Right Winger
17887,2023,Wolves,Premier League,Nélson Semedo,POR,"DF,MF",30-118,1993.0,27.0,27.0,...,10.0,42.0,25.0,978.0,113.0,https://fbref.com/en/players/d04b94db/Nelson-S...,2024Wolves,9.0,€12.00m,Right-Back


In [46]:
name_variation_df =test_df[test_df['Value'].isnull()]
len(name_variation_df)

5520

In [47]:
len(name_variation_df)

5520

In [48]:
test_x = name_variation_df.head(20)
test_y = price_value.head(20)

In [49]:
def find_best_match_with_threshold(name, choices, threshold=80):
    match = process.extractOne(name, choices, scorer=fuzz.token_sort_ratio)
    if match[1] >= threshold:
        return pd.Series([match[0], match[1]])
    else:
        return pd.Series([None, None])

In [50]:
name_variation_df[['MatchedName', 'SimilarityScore']] = name_variation_df['Name'].apply(lambda x: find_best_match_with_threshold(x, price_value['Name'], threshold=80))

C:\Users\russe\AppData\Local\Temp/ipykernel_19952/3162844173.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  name_variation_df[['MatchedName', 'SimilarityScore']] = name_variation_df['Name'].apply(lambda x: find_best_match_with_threshold(x, price_value['Name'], threshold=80))
C:\Users\russe\AppData\Local\Temp/ipykernel_19952/3162844173.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  name_variation_df[['MatchedName', 'SimilarityScore']] = name_variation_df['Name'].apply(lambda x: find_best_match_with

In [51]:
matched = name_variation_df[name_variation_df['SimilarityScore'] >= 88 ]
matched[matched['MatchedName']== "Heung-min Son"]

,Season,Squad_x,Comp_x,Name,Nation_x,Pos_x,Age_x,Born_x,MP_Playing,Starts_Playing,...,Dis_Carries,Rec_Receiving,PrgR_Receiving,Url,Squad_Year,Rk,Value,Position,MatchedName,SimilarityScore
2207,2017,Tottenham,Premier League,Son Heung-min,KOR,FW,25.0,1992.0,37.0,27.0,...,57.0,1067.0,339.0,https://fbref.com/en/players/92e7e919/Son-Heun...,2018Tottenham,3.0,NaN,NaN,Heung-min Son,100.0
4731,2018,Tottenham,Premier League,Son Heung-min,KOR,"FW,MF",26.0,1992.0,31.0,23.0,...,43.0,841.0,201.0,https://fbref.com/en/players/92e7e919/Son-Heun...,2019Tottenham,4.0,NaN,NaN,Heung-min Son,100.0
7229,2019,Tottenham,Premier League,Son Heung-min,KOR,FW,27.0,1992.0,30.0,28.0,...,57.0,853.0,215.0,https://fbref.com/en/players/92e7e919/Son-Heun...,2020Tottenham,6.0,NaN,NaN,Heung-min Son,100.0
9873,2020,Tottenham,Premier League,Son Heung-min,KOR,FW,28.0,1992.0,37.0,36.0,...,50.0,1091.0,266.0,https://fbref.com/en/players/92e7e919/Son-Heun...,2021Tottenham,7.0,NaN,NaN,Heung-min Son,100.0
12570,2021,Tottenham,Premier League,Son Heung-min,KOR,"MF,FW",29.0,1992.0,35.0,35.0,...,46.0,1078.0,242.0,https://fbref.com/en/players/92e7e919/Son-Heun...,2022Tottenham,4.0,NaN,NaN,Heung-min Son,100.0
15149,2022,Tottenham,Premier League,Son Heung-min,KOR,"MF,FW",30.0,1992.0,36.0,33.0,...,55.0,860.0,196.0,https://fbref.com/en/players/92e7e919/Son-Heun...,2023Tottenham,8.0,NaN,NaN,Heung-min Son,100.0
17649,2023,Tottenham,Premier League,Son Heung-min,KOR,FW,31-249,1992.0,24.0,23.0,...,25.0,717.0,228.0,https://fbref.com/en/players/92e7e919/Son-Heun...,2024Tottenham,5.0,NaN,NaN,Heung-min Son,100.0


In [52]:
price_value[price_value['Name']== "Heung-min Son"]

,Name,Value,Season,Position
124,Heung-min Son,€50.00m,2023,Left Winger
3234,Heung-min Son,€50.00m,2022,Left Winger
7546,Heung-min Son,€75.00m,2021,Left Winger
11918,Heung-min Son,€85.00m,2020,Left Winger
16161,Heung-min Son,€64.00m,2019,Left Winger
20291,Heung-min Son,€80.00m,2018,Left Winger
24804,Heung-min Son,€50.00m,2017,Left Winger


In [53]:
matched['Name']=matched['MatchedName']
columns_to_drop = ['Value', 'Position','MatchedName','SimilarityScore']
matched.drop(columns=columns_to_drop, inplace=True)

C:\Users\russe\AppData\Local\Temp/ipykernel_19952/2725251724.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  matched['Name']=matched['MatchedName']
C:\Users\russe\AppData\Local\Temp/ipykernel_19952/2725251724.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  matched.drop(columns=columns_to_drop, inplace=True)


In [54]:
matched.to_csv("footballer_stats_from_fuzzy.csv", index = False)

In [55]:
# threshold_similarity = 80

In [56]:
# matches_above_threshold = df1[df1['SimilarityScore'] >= threshold_similarity]

# Replace names in df1 with corresponding names from df2 for matches above threshold
# df1.loc[matches_above_threshold.index, 'Name'] = matches_above_threshold['MatchedName']

# Drop unnecessary columns
# df1 = df1.drop(['SimilarityScore', 'MatchedName', 'MatchedIndex'], axis=1)

In [57]:
matched = pd.read_csv("footballer_stats_from_fuzzy.csv")

In [60]:
matched[matched['Name']== "Heung-min Son"]

,Season,Squad_x,Comp_x,Name,Nation_x,Pos_x,Age_x,Born_x,MP_Playing,Starts_Playing,...,PrgC_Carries,Final_Third_Carries,CPA_Carries,Mis_Carries,Dis_Carries,Rec_Receiving,PrgR_Receiving,Url,Squad_Year,Rk
378,2017,Tottenham,Premier League,Heung-min Son,KOR,FW,25.0,1992.0,37.0,27.0,...,111.0,46.0,68.0,65.0,57.0,1067.0,339.0,https://fbref.com/en/players/92e7e919/Son-Heun...,2018Tottenham,3.0
784,2018,Tottenham,Premier League,Heung-min Son,KOR,"FW,MF",26.0,1992.0,31.0,23.0,...,105.0,64.0,59.0,60.0,43.0,841.0,201.0,https://fbref.com/en/players/92e7e919/Son-Heun...,2019Tottenham,4.0
1189,2019,Tottenham,Premier League,Heung-min Son,KOR,FW,27.0,1992.0,30.0,28.0,...,90.0,49.0,56.0,66.0,57.0,853.0,215.0,https://fbref.com/en/players/92e7e919/Son-Heun...,2020Tottenham,6.0
1613,2020,Tottenham,Premier League,Heung-min Son,KOR,FW,28.0,1992.0,37.0,36.0,...,88.0,46.0,37.0,59.0,50.0,1091.0,266.0,https://fbref.com/en/players/92e7e919/Son-Heun...,2021Tottenham,7.0
2005,2021,Tottenham,Premier League,Heung-min Son,KOR,"MF,FW",29.0,1992.0,35.0,35.0,...,116.0,68.0,50.0,75.0,46.0,1078.0,242.0,https://fbref.com/en/players/92e7e919/Son-Heun...,2022Tottenham,4.0
2387,2022,Tottenham,Premier League,Heung-min Son,KOR,"MF,FW",30.0,1992.0,36.0,33.0,...,82.0,60.0,31.0,64.0,55.0,860.0,196.0,https://fbref.com/en/players/92e7e919/Son-Heun...,2023Tottenham,8.0
2728,2023,Tottenham,Premier League,Heung-min Son,KOR,FW,31-249,1992.0,24.0,23.0,...,70.0,27.0,42.0,36.0,25.0,717.0,228.0,https://fbref.com/en/players/92e7e919/Son-Heun...,2024Tottenham,5.0


In [61]:
matched

,Season,Squad_x,Comp_x,Name,Nation_x,Pos_x,Age_x,Born_x,MP_Playing,Starts_Playing,...,PrgC_Carries,Final_Third_Carries,CPA_Carries,Mis_Carries,Dis_Carries,Rec_Receiving,PrgR_Receiving,Url,Squad_Year,Rk
0,2017,Alavés,La Liga,Martín Aguirregabiria,ESP,DF,21.0,1996.0,21.0,20.0,...,24.0,13.0,6.0,20.0,10.0,483.0,54.0,https://fbref.com/en/players/355c883a/Martin-A...,2018Alavés,14.0
1,2017,Alavés,La Liga,Ermedin Demirovic,BIH,FW,19.0,1998.0,3.0,2.0,...,2.0,2.0,2.0,4.0,4.0,42.0,11.0,https://fbref.com/en/players/ed79b7d3/Ermedin-...,2018Alavés,14.0
2,2017,Alavés,La Liga,Dani Torres,COL,MF,27.0,1989.0,18.0,14.0,...,4.0,5.0,1.0,14.0,6.0,372.0,13.0,https://fbref.com/en/players/87882adc/Daniel-T...,2018Alavés,14.0
3,2017,Amiens,Ligue 1,Quentin Cornette,FRA,"FW,MF",23.0,1994.0,12.0,6.0,...,15.0,12.0,4.0,16.0,8.0,122.0,20.0,https://fbref.com/en/players/5d42d487/Quentin-...,2018Amiens,13.0
4,2017,Amiens,Ligue 1,Oualid El Hajjam,MAR,DF,26.0,1991.0,26.0,24.0,...,48.0,29.0,3.0,39.0,29.0,593.0,74.0,https://fbref.com/en/players/9f29c204/Oualid-E...,2018Amiens,13.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2758,2023,Wolfsburg,Bundesliga,Joakim Maehle,DEN,DF,26-298,1997.0,23.0,22.0,...,63.0,43.0,7.0,25.0,12.0,811.0,105.0,https://fbref.com/en/players/c459a650/Joakim-M...,2024Wolfsburg,13.0
2759,2023,Wolfsburg,Bundesliga,Cédric Zesiger,SUI,DF,25-263,1998.0,15.0,12.0,...,10.0,7.0,0.0,1.0,2.0,597.0,1.0,https://fbref.com/en/players/2bab583c/Cedric-Z...,2024Wolfsburg,13.0
2760,2023,Wolves,Premier League,Rayan Aït-Nouri,ALG,"DF,MF",22-281,2001.0,24.0,21.0,...,51.0,38.0,7.0,34.0,35.0,736.0,78.0,https://fbref.com/en/players/9b398aea/Rayan-Ai...,2024Wolves,9.0
2761,2023,Wolves,Premier League,Hee-chan Hwang,KOR,"FW,MF",28-047,1996.0,22.0,19.0,...,30.0,19.0,17.0,41.0,25.0,462.0,93.0,https://fbref.com/en/players/169fd162/Hwang-He...,2024Wolves,9.0
